# Optimization using genetic algorithm

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


##  Load the dataset

In [ ]:

data = pd.read_csv(r"C:\Users\amnsh\OneDrive\Desktop\DAA\Dataset_cia2\Bank_Personal_Loan_Modelling.csv")


##  Split the data into features and labels

In [ ]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

##  Convert the labels to one-hot encoding

In [ ]:
y = tf.keras.utils.to_categorical(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## Define the fitness function for the genetic algorithm

In [ ]:

def fitness_function(individual, X_train, y_train, X_test, y_test):
    # Convert the individual into a set of hyperparameters
    hidden_layer_sizes = [32, 64, 128, 256, 512]
    dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
    learning_rates = [0.001, 0.01, 0.1]
    hidden_layer_size = hidden_layer_sizes[individual[0]]
    dropout_rate = dropout_rates[individual[1]]
    learning_rate = learning_rates[individual[2]]
    # Create the model architecture
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(hidden_layer_size, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    # Evaluate the model
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy


## Define the genetic algorithm

In [ ]:

def genetic_algorithm(X_train, y_train, X_test, y_test, population_size=50, generations=20, crossover_probability=0.8, mutation_probability=0.2):
    # Define the population initialization function
    def create_individual():
        hidden_layer_size = np.random.choice([0, 1, 2, 3, 4])
        dropout_rate = np.random.choice([0, 1, 2, 3, 4, 5])
        learning_rate = np.random.choice([0, 1, 2])
        return [hidden_layer_size, dropout_rate, learning_rate]
    # Initialize the population
    population = [create_individual() for _ in range(population_size)]

    # Evaluate the fitness of the initial population
    fitness = [fitness_function(individual, X_train, y_train, X_test, y_test) for individual in population]

    # Iterate over the generations
    for generation in range(generations):
        # Select parents for crossover
        parents = np.random.choice(population, size=population_size, replace=True, p=fitness/sum(fitness))
    
        # Perform crossover
        offspring = []
        for i in range(0, population_size, 2):
            parent1 = parents[i]
            parent2 = parents[i+1]
            if np.random.rand() < crossover_probability:
                # Perform uniform crossover
                child1 = []
                child2 = []
                for j in range(len(parent1)):
                    if np.random.rand() < 0.5:
                        child1.append(parent1[j])
                        child2.append(parent2[j])
                    else:
                        child1.append(parent2[j])
                        child2.append(parent1[j])
                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.append(parent1)
                offspring.append(parent2)
    
    # Mutate the offspring
    for j in range(len(offspring)):
        for k in range(len(offspring[j])):
            if np.random.uniform() < mutation_probability:
                if k == 0:
                    offspring[j][k] = np.random.choice([0, 1, 2, 3, 4])
                elif k == 1:
                    offspring[j][k] = np.random.choice([0, 1, 2, 3, 4, 5])
                else:
                    offspring[j][k] = np.random.choice([0, 1, 2])

    # Evaluate the fitness of the offspring
    offspring_fitness = [fitness_function(individual, X_train, y_train, X_test, y_test) for individual in offspring]

    # Combine the parents and offspring
    combined_population = population + offspring
    combined_fitness = fitness + offspring_fitness

    # Select the next generation
    sorted_indices = np.argsort(combined_fitness)[::-1][:population_size]
    population = [combined_population[i] for i in sorted_indices]
    fitness = [combined_fitness[i] for i in sorted_indices]

    # Print the best accuracy in the current generation

    #print("Generation {}: Best accuracy = {}".format(generation+1, fitness[0]))
    # Return the best individual and its fitness
    best_individual = population[0]
    best_fitness = fitness[0]
    return best_individual, best_fitness
best_individual, best_fitness = genetic_algorithm(X_train, y_train, X_test, y_test, population_size=50, generations=20, crossover_probability=0.8, mutation_probability=0.2)



    

In [ ]:
print("Best individual: {}".format(best_individual))
print("A: {}".format(best_fitness))
